In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gdown==4.6.0

  Attempting uninstall: gdown
    Found existing installation: gdown 5.2.0
    Uninstalling gdown-5.2.0:
      Successfully uninstalled gdown-5.2.0


In [3]:
import gdown
import os

datasets = {
    '/content/data/small_train': [
        ('1ygUBSUi0fZFAU7OcdSygRAixrjwDROor', 'news.tsv'),
        ('18tpIlF46VZOnS-7i8Ic5CTO0kP-E9Twe', 'behaviors.tsv'),
        ('1O7l4lFn_nNtFRKCL0nzW9mU6XS-tEFLE', 'entity_embedding.vec'),
        ('1U4DYtOQOU3FVxKU8ClR65yzUp2ACPi3R', 'relation_embedding.vec')
    ],
    '/content/data/small_dev': [
        ('19fn7ZOPwH7AeWO9l52vQAltN6lI89G7f', 'news.tsv'),
        ('15_x3o8rtqY8EAuPFAEHsjjFN7vZD201c', 'behaviors.tsv'),
        ('13V4JNumyOo5Uu6tSPIsTGdB27_X1HO5K', 'entity_embedding.vec'),
        ('1Z0lQdWAUhAA18l83cyHBlo3WZNNbQZTe', 'relation_embedding.vec')
    ],
    '/content/data/large_full': [
        ('1oOEV6W1wbv2tJco1yhnbkNdwArmlbPkV', 'news.tsv'),
        ('1MSWmDbowqCrBco1Q8YBGJ3UuS_Al9AgD', 'behaviors.tsv')
    ]
}


for folder_path, files in datasets.items():
    os.makedirs(folder_path, exist_ok=True)

    for file_id, file_name in files:
        output_path = os.path.join(folder_path, file_name)

        if not os.path.exists(output_path):
            url = f'https://drive.google.com/uc?id={file_id}'
            try:
                gdown.download(url, output_path, quiet=False)
            except Exception as e:
                print(f"❌ Lỗi tải {file_name}: {e}")
        else:
            print(f"✅ {file_name} đã tồn tại (Skipped)")


Downloading...
From: https://drive.google.com/uc?id=1ygUBSUi0fZFAU7OcdSygRAixrjwDROor
To: /content/data/small_train/news.tsv
100%|██████████| 41.2M/41.2M [00:01<00:00, 22.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=18tpIlF46VZOnS-7i8Ic5CTO0kP-E9Twe
To: /content/data/small_train/behaviors.tsv
100%|██████████| 92.0M/92.0M [00:02<00:00, 36.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O7l4lFn_nNtFRKCL0nzW9mU6XS-tEFLE
To: /content/data/small_train/entity_embedding.vec
100%|██████████| 25.8M/25.8M [00:00<00:00, 59.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1U4DYtOQOU3FVxKU8ClR65yzUp2ACPi3R
To: /content/data/small_train/relation_embedding.vec
100%|██████████| 1.04M/1.04M [00:00<00:00, 5.74MB/s]
Downloading...
From: https://drive.google.com/uc?id=19fn7ZOPwH7AeWO9l52vQAltN6lI89G7f
To: /content/data/small_dev/news.tsv
100%|██████████| 33.5M/33.5M [00:00<00:00, 35.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=15_x3o8rtqY8EAuPFAEHsjjFN7vZD

In [4]:
!git clone https://github.com/hungbuile04/NewsRecSys
%cd NewsRecSys/news-recommendation-llm

Cloning into 'NewsRecSys'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 239 (delta 100), reused 192 (delta 55), pack-reused 0 (from 0)
Receiving objects: 100% (239/239), 418.36 KiB | 2.32 MiB/s, done.
Resolving deltas: 100% (100/100), done.
/content/NewsRecSys/news-recommendation-llm


In [5]:
import os
os.chdir("/content/NewsRecSys/news-recommendation-llm")
os.environ["PYTHONPATH"] = f"{os.getcwd()}/src:{os.getcwd()}"
os.getcwd(), os.environ["PYTHONPATH"]


('/content/NewsRecSys/news-recommendation-llm',
 '/content/NewsRecSys/news-recommendation-llm/src:/content/NewsRecSys/news-recommendation-llm')

In [6]:
import os
import glob

MODE = 'large'    # Thay bằng "small" để test nếu có lỗi thì nhanh phát hiện, large load data lâu

DATA_PATHS = {
    'small': '/content/data/small_train',
    'large': '/content/data/large_full'
}

target_data_dir = DATA_PATHS.get(MODE)
if not target_data_dir or not os.path.exists(target_data_dir):
    raise FileNotFoundError(f"❌ LỖI: Không tìm thấy thư mục '{target_data_dir}'.")

DRIVE_MODEL_DIR = "/content/drive/MyDrive/recsys_model"

def find_latest_checkpoint(base_dir):
    if not os.path.exists(base_dir): return None
    runs = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    if not runs: return None
    latest_run_dir = os.path.join(base_dir, runs[-1])
    checkpoints = glob.glob(os.path.join(latest_run_dir, "checkpoint-*"))
    if not checkpoints: return None
    return max(checkpoints, key=lambda x: int(x.split("-")[-1]))

latest_ckpt = find_latest_checkpoint(DRIVE_MODEL_DIR)

# sửa các tham số batch_size, gradient_accumulation_steps, history_size, epochs trực tiếp tại đây
cmd = f"uv run python src/experiments/train.py data_dir='{target_data_dir}' batch_size=96 gradient_accumulation_steps=2 history_size=20 epochs=1"

if latest_ckpt:
    print(f"🔄 TÌM THẤY CHECKPOINT: {latest_ckpt}")
    print("🚀 Đang tiếp tục train (RESUME)...")
    cmd += f" resume_checkpoint='{latest_ckpt}'"
else:
    print("🆕 Không có checkpoint cũ. BẮT ĐẦU TRAIN MỚI...")

print(f"\nExecuting Command:\n{cmd}\n")
!{cmd}

✅ CHẾ ĐỘ: LARGE
Qs DATA PATH: /content/data/large_full
🔄 TÌM THẤY CHECKPOINT: /content/drive/MyDrive/recsys_model/19-52-03/checkpoint-13000
🚀 Đang tiếp tục train (RESUME)...

Executing Command:
uv run python src/experiments/train.py data_dir='/content/data/large_full' batch_size=96 gradient_accumulation_steps=2 history_size=20 epochs=1 resume_checkpoint='/content/drive/MyDrive/recsys_model/19-52-03/checkpoint-13000'

Using CPython 3.11.3
Creating virtual environment at: .venv
Installed 75 packages in 224ms
2026-01-04 04:17:53,692 INFO Created a temporary directory at /tmp/tmpsfpuk4jm
2026-01-04 04:17:53,693 INFO Writing /tmp/tmpsfpuk4jm/_remote_module_non_scriptable.py
[2026-01-04 04:17:56,131][root][INFO] - Start
/content/NewsRecSys/news-recommendation-llm/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download

In [8]:
import os
import gdown

# Tải dữ liệu test từ drive
output_dir = "/content/data/large_test"
os.makedirs(output_dir, exist_ok=True)

files = {
    "behaviors.tsv": {
        "id": "1ixWnzXv2mG0cbFCmKwB3BFYUgjaRPjH2",
        "output": os.path.join(output_dir, "behaviors.tsv")
    },
    "news.tsv": {
        "id": "1B1ZQ9W3I40tDfiRkSMmXLYnf1h5x-Qga",
        "output": os.path.join(output_dir, "news.tsv")
    }
}

print(f"📂 Đang tải dữ liệu vào: {output_dir}")

for name, info in files.items():
    if not os.path.exists(info["output"]):
        print(f"⬇️ Đang tải {name}...")
        gdown.download(id=info["id"], output=info["output"], quiet=False)
    else:
        print(f"✅ {name} đã tồn tại, bỏ qua.")

print("\n🎉 Đã tải xong dữ liệu Test!")

📂 Đang tải dữ liệu vào: /content/data/large_test
⬇️ Đang tải behaviors.tsv...


Downloading...
From: https://drive.google.com/uc?id=1ixWnzXv2mG0cbFCmKwB3BFYUgjaRPjH2
To: /content/data/large_test/behaviors.tsv
100%|██████████| 1.46G/1.46G [00:17<00:00, 83.7MB/s]


⬇️ Đang tải news.tsv...


Downloading...
From: https://drive.google.com/uc?id=1B1ZQ9W3I40tDfiRkSMmXLYnf1h5x-Qga
To: /content/data/large_test/news.tsv
100%|██████████| 101M/101M [00:02<00:00, 44.8MB/s] 


🎉 Đã tải xong dữ liệu Test!
Kiểm tra thư mục: ['behaviors.tsv', 'news.tsv']


In [15]:
import os
import glob

DRIVE_MODEL_DIR = "/content/drive/MyDrive/recsys_model"
TARGET_RUN_NAME = "04-17-55"

final_model_dir = os.path.join(DRIVE_MODEL_DIR, TARGET_RUN_NAME, "final_model")
ckpt_path = os.path.join(final_model_dir, "pytorch_model.bin")

if not os.path.exists(ckpt_path):
    ckpt_path = os.path.join(final_model_dir, "model.safetensors")

if os.path.exists(ckpt_path):
    print(f"✅ Model Path: {ckpt_path}")

    news_path = "/content/data/large_test/news.tsv"
    behaviors_path = "/content/data/large_test/behaviors.tsv"

    # Tên file zip output
    output_zip = f"prediction_{TARGET_RUN_NAME}.zip"

    cmd = f"python create_submission.py --news_path '{news_path}' --behaviors_path '{behaviors_path}' --checkpoint_path '{ckpt_path}' --output_path 'prediction.txt'"

    print(f"\nExecuting: {cmd}")
    !{cmd}

    if os.path.exists("prediction.txt"):
        print("\n📦 Đang nén file zip...")
        !zip {output_zip} prediction.txt

        print(f"\n💾 Đang lưu vào Drive: {DRIVE_MODEL_DIR}/{output_zip}")
        !cp {output_zip} {DRIVE_MODEL_DIR}/{output_zip}

        print("\n🎉 HOÀN TẤT! Bạn hãy vào Drive kiểm tra folder recsys_model nhé.")
    else:
        print("\n❌ Vẫn lỗi: Không thấy file prediction.txt được tạo ra.")
else:
    print(f"❌ Lỗi: Không tìm thấy model tại {ckpt_path}")

✅ Model Path: /content/drive/MyDrive/recsys_model/04-17-55/final_model/pytorch_model.bin

Executing: python create_submission.py --news_path '/content/data/large_test/news.tsv' --behaviors_path '/content/data/large_test/behaviors.tsv' --checkpoint_path '/content/drive/MyDrive/recsys_model/04-17-55/final_model/pytorch_model.bin' --output_path 'prediction.txt'
🔄 Loading model from: /content/drive/MyDrive/recsys_model/04-17-55/final_model/pytorch_model.bin
2026-01-04 05:18:11.909906: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-04 05:18:11.928205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767503891.949869   17329 